# Cerebro Subintenciones pagos

In [2]:
import numpy as np
import pandas as pd
import sklearn
import nltk

###### Para este cerebro solo voy a tomar las oraciones que esten clasificadas como pagos, para evitar el ruido del resto

In [3]:
df_textos=pd.read_csv('dataIntents06depurado.csv', sep=';', encoding = "ISO-8859-1")
df_textos = df_textos[df_textos['intencion']=='pagos']

df_textos

df_textos = df_textos.reset_index()

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
corpus=df_textos.oracion

import json
# Opening JSON file 
f = open('stopwords.json',) 
stopwords = json.load(f)['words']

vectorizer = CountVectorizer(binary=False)#, stop_words= stopwords)
X = vectorizer.fit_transform(corpus)

## Vocabulario

In [5]:
vocabulario=vectorizer.get_feature_names()

In [6]:
# Cuántas palabras distintas hay en el Vocabulario?
len(vocabulario)

162

In [7]:
X.toarray()
df_X=pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
df_X

,abonar,aceptan,adelantado,ahí,al,algun,algunas,algún,aparezco,arancel,...,todo,todos,un,usar,varias,varios,veraz,vez,visa,voy
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
132,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
133,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
134,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
df=df_textos.join(df_X)
df = df.drop(['index'], axis=1) #Al resetear index al principio, me quedo esta de mas
df

,oracion,intencion,subIntencion,abonar,aceptan,adelantado,ahí,al,algun,algunas,...,todo,todos,un,usar,varias,varios,veraz,vez,visa,voy
0,cuánto cuesta la carrera,pagos,precio,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,cuánto cuesta,pagos,precio,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,cómo se pude pagar,pagos,medios,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,cuánto cuesta estudiar,pagos,precio,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,disculpe quería preguntar cuánto cuesta,pagos,precio,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,que forma de pagos tiene la carrera de ciencia...,pagos,medios,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
132,se puede pagar por materia la carrea de ia,pagos,medios,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
133,hay que pagar inscripción en la carrera de ia,pagos,precios,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
134,los exámenes tienen costo,pagos,precios,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Modelizacion

In [9]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV

In [10]:
X=df.drop(axis=1,columns=['oracion','intencion','subIntencion'])
y=df['subIntencion']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=123)

#### REGRESION LOGISTICA

In [11]:
print("Modelo: Regresión Logística sin regularizacion")

from sklearn.linear_model import LogisticRegression

# Creamos el modelo con los parámetros que no cambiarán, observe que no le pasamos el valor de C de regulrización
RLog=LogisticRegression(penalty='none', max_iter=10000, tol=0.0001, multi_class='ovr',)

# Armamos el diccionario con el  nombre y valores para los Hiperparámetros
parametros_RLog = {'C':[1]}

# Armamos el GridSearchCV
grid_RLog = GridSearchCV(estimator = RLog,scoring = 'accuracy',param_grid = parametros_RLog, cv = 5, verbose = 1,
                        n_jobs = -1)

# Lo entrenamos en el Train Set (él lo dividirá internamente y en cada fold estarán en ValTrain y ValTest)
grid_RLog.fit(X_train, y_train);
# Listo!

# Obtenemos el mejor AC 
AC_RLog_best=grid_RLog.best_score_
print('Mejor accuracy: ' + str(round(AC_RLog_best,4)))

C_RLog_best=grid_RLog.best_params_ 
print('Mejor C: ' + str(C_RLog_best))

Modelo: Regresión Logística sin regularizacion
Fitting 5 folds for each of 1 candidates, totalling 5 fits


C:\Users\Mateio\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Mejor accuracy: 0.8515
Mejor C: {'C': 1}


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    4.3s finished


#### REGRESION LOGISTICA CON REGULARIZACION RIDGE

In [12]:
print("Modelo: Regresión Logística con Regularización Ridge")

from sklearn.linear_model import LogisticRegression

# Creamos el modelo con los parámetros que no cambiarán, observe que no le pasamos el valor de C de regulrización
Log_Ridge=LogisticRegression(penalty='l2', max_iter=20000, multi_class='ovr') 

# Armamos el diccionario con el  nombre y valores para los Hiperparámetros
parametros_Ridge = {'C':[1e-6,1e-5,1e-4,0.001,0.01,0.1,1,10,100,1000,10000]}

# Armamos el GridSearchCV
grid_Ridge = GridSearchCV(estimator = Log_Ridge,param_grid = parametros_Ridge,scoring = 'accuracy', cv = 5, verbose = 1,
                        n_jobs = -1)

# Lo entrenamos en el Train Set (él lo dividirá internamente y en cada fold estarán en ValTrain y ValTest)
grid_Ridge.fit(X_train, y_train)
# Listo!

# Obtenemos el mejor AC 
AC_Ridge_best=grid_Ridge.best_score_
print("Mejor accuracy: " + str(round(AC_Ridge_best,4)))

C_Ridge_best=grid_Ridge.best_params_ 
print('Mejor C: ' + str(C_Ridge_best))

C:\Users\Mateio\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Modelo: Regresión Logística con Regularización Ridge
Fitting 5 folds for each of 11 candidates, totalling 55 fits
Mejor accuracy: 0.8515
Mejor C: {'C': 1000}


[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    1.0s finished


##### REGRESION LOGISTICA CON REGULARIZACION LASSO

In [13]:
print("Modelo: Regresión Logística con Regularización Lasso")

from sklearn.linear_model import LogisticRegression

# Creamos el modelo con los parámetros que no cambiarán, observe que no le pasamos el valor de C de regulrización
Log_Lasso=LogisticRegression(penalty='l1', solver= 'liblinear', max_iter=10000, tol=0.0001, multi_class='ovr') 

# Armamos el diccionario con el  nombre y valores para los Hiperparámetros
parametros_Lasso = {'C':[1e-6,1e-5,1e-4,0.001,0.01,0.1,1,10,100,1000]}

# Armamos el GridSearchCV
grid_Lasso = GridSearchCV(estimator = Log_Lasso,param_grid = parametros_Lasso,scoring = 'accuracy', cv = 5, verbose = 1,
                        n_jobs = -1)

# Lo entrenamos en el Train Set (él lo dividirá internamente y en cada fold estarán en ValTrain y ValTest)
grid_Lasso.fit(X_train, y_train);
# Listo!

# Obtenemos el mejor AC 
AC_Lasso_best=grid_Lasso.best_score_
print('Mejor accuracy: ' + str(round(AC_Lasso_best,4)))

C_Lasso_best=grid_Lasso.best_params_ 
print('Mejor C: ' + str(C_Lasso_best))

C:\Users\Mateio\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Modelo: Regresión Logística con Regularización Lasso
Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Done  43 out of  50 | elapsed:    0.1s remaining:    0.0s


Mejor accuracy: 0.8693
Mejor C: {'C': 1000}


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   19.8s finished


##### RANDOM FOREST

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

#Creo el modelo
rf=RandomForestClassifier(n_jobs=-1,random_state=123)

#Le paso el modelo al grid search con los parametros
grid_rf= GridSearchCV(estimator=rf, param_grid={'n_estimators':[100,150,200,250,300,400,500]}, cv=5,n_jobs=-1, return_train_score=True)

#Entreno el modelo
grid_rf.fit(X_train, y_train)

#Obtengo el accuracy del mejor
AC_rf_best=grid_rf.best_score_
print('Mejor accuracy: ' + str(round(AC_rf_best,4)))
    
NroArboles_rf_best = grid_rf.best_params_
print('Mejor Nro arboles: ' + str(NroArboles_rf_best))


C:\Users\Mateio\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


Mejor accuracy: 0.8693
Mejor Nro arboles: {'n_estimators': 300}


In [15]:
resultados_RLog=pd.DataFrame(grid_RLog.cv_results_)
resultados_RLog = resultados_RLog[['params','mean_test_score']]
resultados_RLog['modelo'] = 'Reg Log sin reg'
resultados_RLog

resultados_rf=pd.DataFrame(grid_rf.cv_results_)
resultados_rf = resultados_rf[['params','mean_test_score']]
resultados_rf['modelo'] = 'Random Forest'
resultados_rf

resultados_Ridge=pd.DataFrame(grid_Ridge.cv_results_)
resultados_Ridge = resultados_Ridge[['params','mean_test_score']]
resultados_Ridge['modelo'] = 'RLog con Ridge'
resultados_Ridge

resultados_Lasso=pd.DataFrame(grid_Lasso.cv_results_)
resultados_Lasso = resultados_Lasso[['params','mean_test_score']]
resultados_Lasso['modelo'] = 'RLog con Lasso'
resultados_Lasso

resultados_todos = pd.concat([resultados_RLog, resultados_rf, resultados_Ridge, resultados_Lasso])

print('El mejor modelo es:')
resultados_todos.sort_values('mean_test_score', ascending=False).head(4)



El mejor modelo es:


,params,mean_test_score,modelo
4,{'n_estimators': 300},0.869264,Random Forest
5,{'n_estimators': 400},0.869264,Random Forest
6,{'n_estimators': 500},0.869264,Random Forest
9,{'C': 1000},0.869264,RLog con Lasso


##### Generemos el "cerebrito" con este valor entrenando en todo el dataset.

In [16]:
mejor_parametro = 1000
modelo_produccion=LogisticRegression(penalty='l2', C=mejor_parametro, max_iter=20000, tol=0.0001, multi_class='ovr')
modelo_produccion.fit(X, y)  # Entrenamos con .fit

LogisticRegression(C=1000, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=20000,
                   multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [17]:
import pickle

nombre_archivo='cerebro_SubintencionPagos.sav'

pickle.dump(modelo_produccion, open(nombre_archivo, 'wb'))

In [18]:
vectorizer_filename = 'vectorizador_SubintencionPagos.sav'
pickle.dump(vectorizer, open(vectorizer_filename, 'wb'))

## Probando

Ingresemos una oración y veamos qué pronostica:

In [19]:
nueva_oracion='quiero pagar con tarjeta'
#Hay que vectorizarla:
X_nueva_oracion = vectorizer.transform([nueva_oracion])
#X_nueva_oracion

# Pronosticamos:
intencion_pronsticada=modelo_produccion.predict(X_nueva_oracion)
intencion_pronsticada

array(['medios'], dtype=object)